In [12]:
from __future__ import print_function
import sys
import os
import numpy as np
import torch

#from pytorch_edgeml.graph.protoNN import ProtoNN
#from pytorch_edgeml.trainer.protoNNTrainer import ProtoNNTrainer
from protoNN_local import ProtoNN
from protoNNTrainer_local import ProtoNNTrainer
import utils_local as utils
import helpermethods as helper

## USPS Data
It is assumed that the USPS data has already been downloaded and set up with the help of `fetch_usps.py` and is placed in the `./usps10` subdirectory.

In [13]:
# Load data
DATA_DIR = './usps10'
train, test = np.load(DATA_DIR + '/train.npy'), np.load(DATA_DIR + '/test.npy')
x_train, y_train = train[:, 1:], train[:, 0]
x_test, y_test = test[:, 1:], test[:, 0]

numClasses = max(y_train) - min(y_train) + 1
numClasses = max(numClasses, max(y_test) - min(y_test) + 1)
numClasses = int(numClasses)

y_train = helper.to_onehot(y_train, numClasses)
y_test = helper.to_onehot(y_test, numClasses)
dataDimension = x_train.shape[1]
numClasses = y_train.shape[1]

## Model Parameters

Note that ProtoNN is very sensitive to the value of the hyperparameter $\gamma$, here stored in valiable GAMMA. If GAMMA is set to None, median heuristic will be used to estimate a good value of $\gamma$ through the helper.getGamma() method. This method also returns the corresponding W and B matrices which should be used to initialize ProtoNN (as is done here).

In [14]:
PROJECTION_DIM = 60
NUM_PROTOTYPES = 60
REG_W = 0.000005
REG_B = 0.0
REG_Z = 0.00005
SPAR_W = 0.8
SPAR_B = 1.0
SPAR_Z = 1.0
LEARNING_RATE = 0.05
NUM_EPOCHS = 200
BATCH_SIZE = 32
GAMMA = 0.0015

In [15]:
W, B, gamma = helper.getGamma(GAMMA, PROJECTION_DIM, dataDimension,
                       NUM_PROTOTYPES, x_train)

In [16]:
protoNNObj = ProtoNN(dataDimension, PROJECTION_DIM, NUM_PROTOTYPES, numClasses,
                     gamma, W=W, B=B)
protoNNTrainer = ProtoNNTrainer(protoNNObj, REG_W, REG_B, REG_Z, SPAR_W, SPAR_B, SPAR_W,
                                LEARNING_RATE, lossType='xentropy')

Using x-entropy loss


In [17]:
protoNNTrainer.train(BATCH_SIZE, NUM_EPOCHS, x_train, x_test, y_train, y_test, printStep=600, valStep=10)

Epoch 0 batch 0 loss 13.823788 acc 0.125000
Epoch 1 batch 0 loss 1.719323 acc 0.500000
Epoch 2 batch 0 loss 1.131474 acc 0.687500
Epoch 3 batch 0 loss 0.733530 acc 0.843750
Epoch 4 batch 0 loss 0.612357 acc 0.906250
Epoch 5 batch 0 loss 0.562658 acc 0.875000
Epoch 6 batch 0 loss 0.512034 acc 0.906250
Epoch 7 batch 0 loss 0.479914 acc 0.937500
Epoch 8 batch 0 loss 0.447306 acc 0.937500
Epoch 9 batch 0 loss 0.422184 acc 0.937500
Validation accuracy: 0.871450
Epoch 10 batch 0 loss 0.402824 acc 0.937500
Epoch 11 batch 0 loss 0.388065 acc 0.937500
Epoch 12 batch 0 loss 0.376297 acc 0.937500
Epoch 13 batch 0 loss 0.370000 acc 0.937500
Epoch 14 batch 0 loss 0.366555 acc 0.906250
Epoch 15 batch 0 loss 0.361238 acc 0.906250
Epoch 16 batch 0 loss 0.359067 acc 0.937500
Epoch 17 batch 0 loss 0.359318 acc 0.937500
Epoch 18 batch 0 loss 0.358350 acc 0.937500
Epoch 19 batch 0 loss 0.365270 acc 0.937500
Validation accuracy: 0.887892
Epoch 20 batch 0 loss 0.367148 acc 0.937500
Epoch 21 batch 0 loss 0.3

Epoch 174 batch 0 loss 0.296315 acc 0.937500
Epoch 175 batch 0 loss 0.296138 acc 0.937500
Epoch 176 batch 0 loss 0.295759 acc 0.937500
Epoch 177 batch 0 loss 0.295540 acc 0.937500
Epoch 178 batch 0 loss 0.293428 acc 0.937500
Epoch 179 batch 0 loss 0.293279 acc 0.937500
Validation accuracy: 0.912307
Epoch 180 batch 0 loss 0.293060 acc 0.937500
Epoch 181 batch 0 loss 0.292918 acc 0.937500
Epoch 182 batch 0 loss 0.292046 acc 0.937500
Epoch 183 batch 0 loss 0.291692 acc 0.937500
Epoch 184 batch 0 loss 0.291389 acc 0.937500
Epoch 185 batch 0 loss 0.290808 acc 0.937500
Epoch 186 batch 0 loss 0.290092 acc 0.937500
Epoch 187 batch 0 loss 0.289793 acc 0.937500
Epoch 188 batch 0 loss 0.289463 acc 0.937500
Epoch 189 batch 0 loss 0.289202 acc 0.937500
Validation accuracy: 0.914798
Epoch 190 batch 0 loss 0.288935 acc 0.937500
Epoch 191 batch 0 loss 0.288602 acc 0.937500
Epoch 192 batch 0 loss 0.288231 acc 0.937500
Epoch 193 batch 0 loss 0.288398 acc 0.937500
Epoch 194 batch 0 loss 0.287257 acc 0.93

## Evaluation

In [18]:
 x_, y_= torch.Tensor(x_test), torch.Tensor(y_test)
logits = protoNNObj.forward(x_)
_, predictions = torch.max(logits, dim=1)
_, target = torch.max(y_, dim=1)
acc, count = protoNNTrainer.accuracy(predictions, target)
W, B, Z, gamma  = protoNNObj.getModelMatrices()
matrixList = [W, B, Z]
matrixList = [x.detach().numpy() for x in matrixList]
sparcityList = [SPAR_W, SPAR_B, SPAR_Z]
nnz, size, sparse = helper.getModelSize(matrixList, sparcityList)
print("Final test accuracy", acc)
print("Model size constraint (Bytes): ", size)
print("Number of non-zeros: ", nnz)
nnz, size, sparse = helper.getModelSize(matrixList, sparcityList,
                                        expected=False)
print("Actual model size: ", size)
print("Actual non-zeros: ", nnz)

Final test accuracy tensor(0.9153, dtype=torch.float64)
Model size constraint (Bytes):  78240
Number of non-zeros:  19560
Actual model size:  78240
Actual non-zeros:  16368
